In [1]:
import psycopg2
import pgdb

In [2]:
from collections import Counter
import folium
from folium.plugins import MarkerCluster

In [3]:
%matplotlib inline
import numpy as npcd
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import mpld3
from mpld3 import plugins, utils
mpl.style.use('ggplot')
mpld3.enable_notebook()

In [4]:
from descartes import PolygonPatch
import math
import json

In [5]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5

In [6]:
from collections import Counter

In [7]:
geocon = psycopg2.connect(database = 'geo', user = 'larsj')

In [8]:
ngramcon = psycopg2.connect(database='ngram_trends')

In [314]:
geocon.rollback()

In [9]:
with geocon.cursor() as cur:
    cur.execute("select * from stedsinfo limit 1")
    colnames = [desc[0] for desc in cur.description]
colnames
    

/home/larsj/anacondapython3/lib/python3.5/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/home/larsj/anacondapython3/lib/python3.5/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):


['ogc_fid', 'enh_snavn', 'enh_navntype', 'kode', 'wkb_geometry']

In [12]:
def sqlps(db, sql):
    with psycopg2.connect(database=db) as con:
        cur =con.cursor()
        cur.execute(sql)
        return cur.fetchall()
    
def sqlpg(db, sql, params = ()):
    with pgdb.connect(database=db) as con:
        cur =con.cursor()
        cur.execute(sql, params)
        return cur.fetchall()

In [26]:
sqlcon(geocon, "select similarity('hallo', 'allo')")

[(0.375,)]

In [27]:
sqlcon(geocon, "select enh_snavn, similarity(enh_snavn, 'trondheim') as sim from stedsinfo where enh_snavn %% 'trondheim' \
order by sim desc limit 20")

[('Trondheim', 1.0),
 ('Trondheim S', 0.833333),
 ('Trondheim tinghus', 0.588235),
 ('Trondheim bymark', 0.588235),
 ('Rondheim', 0.583333),
 ('Rondheim', 0.583333),
 ('Trondeim', 0.583333),
 ('Tronheim', 0.583333),
 ('Tronheim', 0.583333),
 ('Trondheimsleia', 0.5625),
 ('Trondheimsleia', 0.5625),
 ('Trondheimsleia', 0.5625),
 ('Trondheim fengsel', 0.555556),
 ('Trondheim sentrum', 0.555556),
 ('Trondheim kommune', 0.555556),
 ('Tronsheim', 0.538462),
 ('Trandheim', 0.538462),
 ('Tronkheim', 0.538462),
 ('Trandheim', 0.538462),
 ('Trondheimsveien', 0.529412)]

In [251]:
sqlcon(geocon, "select distinct enh_navntype from stedsinfo \
order by enh_navntype")

[('Adm. bydel',),
 ('Adm. tettsted',),
 ('Allmenning',),
 ('Ankringsplass',),
 ('Annen adm. Inndeling',),
 ('Annen bygning for religiøse aktiviteter',),
 ('Annen kulturdetalj',),
 ('Ås',),
 ('Badeplass',),
 ('Båe',),
 ('Båe i sjø',),
 ('Båke',),
 ('Bakke',),
 ('Bakke i sjø',),
 ('Banke',),
 ('Banke i sjø',),
 ('Barnehage',),
 ('Bekk',),
 ('Bergverk (underjord./dagbrudd)',),
 ('Boligblokk',),
 ('Boligfelt',),
 ('Bomstasjon',),
 ('Borettslag',),
 ('Botn',),
 ('Bru',),
 ('Bruk (gardsbruk)',),
 ('Brygge',),
 ('Busstopp',),
 ('By',),
 ('Bydel',),
 ('Bygdelag (bygd)',),
 ('Bygg for jordbruk, fiske og fangst',),
 ('Campingplass',),
 ('Dal',),
 ('Dalføre',),
 ('Dam',),
 ('Del av innsjø',),
 ('Dyp (havdyp)',),
 ('Egg',),
 ('Eid',),
 ('Eid i sjø',),
 ('Eiendommer',),
 ('Elv',),
 ('Elvemel',),
 ('Enebolig/mindre boligbygg (villa)',),
 ('Eng',),
 ('Fabrikk',),
 ('Farled/Skipslei',),
 ('Fengsel',),
 ('Ferjekai ',),
 ('Ferjestrekning',),
 ('Fiskeoppdrettsanlegg',),
 ('Fiskeplass',),
 ('Fjell',),
 ('

In [280]:
sqlcon(geocon, "select enh_navntype, count(enh_navntype) as antall from stedsinfo \
where enh_navntype in ('By','Dal','Bydel','Elv', 'Tettsted','Fjell','Fjellområde','Fjord','Flyplass','Foss','Hotell','Innsjø','Kanal', 'Kirke', \
'Kommune','Øy','Skog', 'Strand','Sykehus') group by enh_navntype order by antall desc")

[('Dal', 42684),
 ('Fjell', 27570),
 ('Elv', 12329),
 ('Skog', 8522),
 ('Foss', 3772),
 ('Fjord', 2376),
 ('Kirke', 1903),
 ('Øy', 1408),
 ('Strand', 1068),
 ('Tettsted', 952),
 ('Innsjø', 559),
 ('Kommune', 470),
 ('Bydel', 413),
 ('Fjellområde', 390),
 ('Hotell', 372),
 ('Flyplass', 170),
 ('By', 125),
 ('Sykehus', 110),
 ('Kanal', 67)]

In [258]:
sqlcon(geocon, "select * from stedsinfo \
 where enh_snavn ilike 'oslo'")

[(80083,
  'Oslo',
  'By',
  'Oslo',
  '0101000020E61000004E9D47C5FF7D25405E126745D4F44D40')]

In [253]:
sqlcon(geocon, "select * from stedsinfo where enh_snavn like 'oslo' \
and enh_navntype in ('Tettsted','Adm. bydel','Adm. tettsted') limit 50")

[]

In [31]:
import html
myhtml = dict(map(lambda x: (x[1], x[0]), list(html.entities.entitydefs.items())))
n2c = html.entities.name2codepoint
mycodes = {x:n2c[myhtml[x]] for x in myhtml}

In [30]:
mycodes

{'"': 34,
 '&': 38,
 '<': 60,
 '>': 62,
 '\xa0': 160,
 '¡': 161,
 '¢': 162,
 '£': 163,
 '¤': 164,
 '¥': 165,
 '¦': 166,
 '§': 167,
 '¨': 168,
 '©': 169,
 'ª': 170,
 '«': 171,
 '¬': 172,
 '\xad': 173,
 '®': 174,
 '¯': 175,
 '°': 176,
 '±': 177,
 '²': 178,
 '³': 179,
 '´': 180,
 'µ': 181,
 '¶': 182,
 '·': 183,
 '¸': 184,
 '¹': 185,
 'º': 186,
 '»': 187,
 '¼': 188,
 '½': 189,
 '¾': 190,
 '¿': 191,
 'À': 192,
 'Á': 193,
 'Â': 194,
 'Ã': 195,
 'Ä': 196,
 'Å': 197,
 'Æ': 198,
 'Ç': 199,
 'È': 200,
 'É': 201,
 'Ê': 202,
 'Ë': 203,
 'Ì': 204,
 'Í': 205,
 'Î': 206,
 'Ï': 207,
 'Ð': 208,
 'Ñ': 209,
 'Ò': 210,
 'Ó': 211,
 'Ô': 212,
 'Õ': 213,
 'Ö': 214,
 '×': 215,
 'Ø': 216,
 'Ù': 217,
 'Ú': 218,
 'Û': 219,
 'Ü': 220,
 'Ý': 221,
 'Þ': 222,
 'ß': 223,
 'à': 224,
 'á': 225,
 'â': 226,
 'ã': 227,
 'ä': 228,
 'å': 229,
 'æ': 230,
 'ç': 231,
 'è': 232,
 'é': 233,
 'ê': 234,
 'ë': 235,
 'ì': 236,
 'í': 237,
 'î': 238,
 'ï': 239,
 'ð': 240,
 'ñ': 241,
 'ò': 242,
 'ó': 243,
 'ô': 244,
 'õ': 245,
 'ö': 24

In [33]:
def enheter(sted, limit=50):
    enheter = ('By','Dal','Bydel','Elv', 'Tettsted','Fjell', 
               'Fjellområde','Fjord','Flyplass','Foss','Hotell','Innsjø','Kanal', 'Kirke',
                'Kommune','Øy','Skog', 'Strand','Sykehus')
    
    rows = sqlcon(geocon, "select enh_snavn ||', '||enh_navntype || ' i ' || kode, ST_AsLatLonText( wkb_geometry, 'D.DDDD') from \
    stedsinfo where enh_snavn ilike %s limit %s", (sted, limit))
    coords = []
    for row in rows:
        x = row[1].split()
        #print(x)
        coords.append([changecode(row[0]), x])
    return coords

def changecode(text, codes = mycodes):
    new = []
    for c in text:
        if c in codes:
            #print(c, codes[c])
            new.append('&#%s;'%(codes[c],))
        else:
            new.append(c)
    return ''.join(new)
    


def kart(sted, limit=50):
    import numpy as np
    coords = enheter(sted, int(limit/2))
    kartsnitt = folium.Map(location=[62, 4], zoom_start = 4)
    if coords != []:
        locs = []
        pops = []
        #beregn gjennomsnittlig lengde og bredde og start kartet derfra
        lat = np.mean([float(crd[1][0]) for crd in coords])
        lon = np.mean([float(crd[1][1]) for crd in coords])
        kartsnitt = folium.Map(location=[lat, lon], zoom_start = 6)
        for sted in coords:
            #folium.Marker(sted[1], popup = sted[0], icon=folium.Icon(icon='cloud')).add_to(kartsnitt)
            locs.append(sted[1])
            pops.append(sted[0])
        kartsnitt.add_children(MarkerCluster(locs, popups=pops))
        #print(locs)
    return kartsnitt

def leggtil(kartsnitt, sted, limit=50):
    import numpy as np
    coords = enheter(sted, int(limit/2))
    if coords != []:
        locs = []
        pops = []
        for sted in coords:
            #folium.Marker(sted[1], popup = sted[0], icon=folium.Icon(icon='cloud')).add_to(kartsnitt)
            locs.append(sted[1])
            pops.append(sted[0])
        kartsnitt.add_children(MarkerCluster(locs, popups=pops))
        #print(locs)
    return kartsnitt

In [34]:

kart('oslo%').save('testkart.html')


In [361]:
geocon.rollback()

In [35]:
coords = koordinater('os')
lat = np.mean([crd[1][0] for crd in coords])
lon = np.mean([crd[1][1] for crd in coords])
map_1 = folium.Map(location=[lat, lon], zoom_start = 5)
for sted in coords:
    folium.Marker(sted[1], popup=sted[0]).add_to(map_1)
map_1

NameError: name 'koordinater' is not defined

In [14]:
import json

geo_source = GeoJSONDataSource(geojson=json.dumps(coords))
p = figure()
p.circle(x='x', y='y', alpha=0.9, source= geo_source)
show(p)

NameError: name 'GeoJSONDataSource' is not defined

In [15]:
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)

map_options = GMapOptions(lat=30.29, lng=-97.73, map_type="roadmap", zoom=11)

plot = GMapPlot(
    x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options, title="Austin"
)

source = ColumnDataSource(
    data=dict(
        lat=[30.29, 30.20, 30.29],
        lon=[-97.70, -97.74, -97.78],
    )
)

circle = Circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, line_color=None)
plot.add_glyph(source, circle)

plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())

show(plot)

NameError: name 'show' is not defined

In [16]:
def sqldb(db, sql, params = ()):
    """Kjør sql mot databasen angitt db, som skal være et databasenavn i Postgres. 
    Tilkobling med psycopg2. Argumentet sql er selve sql-uttrykket, 
    mens params er parametrene, som trygt kan utelates om det ikke er noe %s i sql."""
    
    con = psycopg2.connect(database=db)
    with con.cursor() as cur:
        cur.execute(sql, params)
        res = cur.fetchall()
    con.close()
    return res
    
def sqlcon(con, sql, params = ()):
    """Kjør sql mot databasen angitt db, som skal være et databasenavn i Postgres. 
    Tilkobling med psycopg2. Argumentet sql er selve sql-uttrykket, 
    mens params er parametrene, som trygt kan utelates om det ikke er noe %s i sql."""
    
    with con.cursor() as cur:
        cur.execute(sql, params)
        res = cur.fetchall()
    return res
    
def finddb(urn, antall = 9):
    """Finn ut hvilken fulltekst-database URNen befinner seg i. 
    Alle fulltekstbasene er på formen ft-N der N går fra 0 og oppover. Parameter antall
    spesifiserer hvor mange databasenavn som skal genereres. Pr. nå er det 9, som er default."""
        
    dbs = ['ft-%s' % (str(i),) for i in range(antall)]
    hit = False
    i = 0
    while not(hit) and i < len(dbs):
        res = sqlexe(dbs[i],'select urn from unigram where urn = %s limit 1',(urn,))
        hit = res != []
        if not(hit):
            i += 1
    if i < len(dbs):
        result = dbs[i]
    else:
        result = 'notfound'
        
    return result

def db_find(urn):
    """Finn ut hvilken fulltekst-database URNen befinner seg i. 
    Alle fulltekstbasene er på formen ft-N der N går fra 0 og oppover. Parameter antall
    spesifiserer hvor mange databasenavn som skal genereres. Pr. nå er det 9, som er default."""
        
    row =  sqldb('mods', 'select dbid from mods_new where urn = %s', (urn,))
    if row != []:
        res = 'ft-%s' % (row[0][0],)
    else:
        res = 'notfound'
    return res

def following_urn(urn, signs=('.',':','?','!')):
    """Finn alle ordene som følger etter et punktum i boka med urn urn,
    summert med frekvens og returnert som dictionary"""
    
    db = db_find(urn)
    after_period=[]
    if db != 'notfound':
        after_period = sqldb(db, "select a.word, count(a.word) from ft as a, ft as b \
        where a.urn = %s and a.urn = b.urn and b.seq = a.seq - 1 and b.word in %s \
        group by a.word", (urn, signs))
    return {row[0]:row[1] for row in after_period}


def following_bi(word, signs=('.',':','?','!'), lang='nob'):
    """Finn alle ordene som følger etter et punktum i bigrambasen,
    summert med frekvens og returnert som dictionary"""
    
    after_period=[]
    after_period = sqldb('ngram_trends', "select second, sum(freq) from bigram \
        where first in %s and second = %s and lang=%s \
        group by second", (signs, word, lang))
    totals = sqldb('ngram_trends', "select")
    return {row[0]:row[1] for row in after_period}

def compare(word, signs=('.',':','?','!'), lang='nob'):
    """Sammenlign bigram etterfølger med unigram"""
    
    after_period=[]
    after_period = sqldb('ngram_trends', "select a.second, sum(b.freq)/sum(a.freq) from bigram as a, unigram as b \
        where a.first in %s and a.second = %s and a.lang = %s and a.lang=b.lang and b.first = a.second \
        group by second", (signs, word, lang))
    totals = sqldb('ngram_trends', "select")
    return {row[0]:float(row[1]) for row in after_period}

def compare_all(signs=('.',':','?','!'), lang='nob'):
    """Sammenlign bigram etterfølger med unigram"""
    
    after_period=[]
    after_period = sqldb('ngram_trends', "select b.first, sum(b.freq)/sum(a.freq) from bigram as a, unigram as b \
        where a.first in %s and a.lang = %s and a.lang=b.lang and b.first = a.second and lower(b.first) != b.first \
        group by b.first", (signs, lang))
    totals = sqldb('ngram_trends', "select")
    return {row[0]:float(row[1]) for row in after_period}

def unigram(urn):
    """Hent ut frekvenslisten fra boka som et dictionary"""
    
    res = sqldb('ngram_book', 'select word, freq from unigram where urn = %s', (urn,))
    return {row[0]:row[1] for row in res}


def find_books(navn = "%", tittel = "%", lang='nob', limit = 40):
    """Find URN for books based on a list of titles and names"""
    
    import psycopg2

    sql = "select * from mods_new where creator ilike %s and title ilike %s and lang like %s limit %s" 
    #print(sql)
    with psycopg2.connect(database='mods') as con:
        cur = con.cursor()
        cur.execute(sql, (navn, tittel, lang, limit)) 
        x = cur.fetchall()
        urns = [row for row in x]
    return urns

def smooth_values(values, smooth=3):
    """Glatt ut en array med verdier - returner en liste med samme lengde"""
    for i in range(len(values)):
        values[i] = sum(values[max(0, i-smooth):min(i+smooth, len(values))])/(2*smooth + 1)
    return values

In [17]:
def gammelnytt():
    """Sjekk om et substantiv er gammelt eller nytt"""
    rows = sqldb('ngram_trends', "select b.first, avg(a.year) from unigram_year as a, unigram as b where b.id = a.id and lower(b.first) != b.first group by b.first")
    return {row[0]: float(row[1]) for row in rows}

In [18]:
unigammel = gammelnytt()

ProgrammingError: relation "unigram_year" does not exist
LINE 1: select b.first, avg(a.year) from unigram_year as a, unigram ...
                                         ^


In [ ]:
Counter(unigammel).most_common(20)

In [ ]:
yeardb = sqlcon(ngramcon, "select year, sum(freq) from unigram_year group by year")

In [ ]:
years = {key:int(val) for (key,val) in yeardb}


In [ ]:
years

In [ ]:
ngramcon.rollback()

In [19]:
def trend(word, lang='nob'):
    res = sqldb('ngram_trends', "select json from unigram where first=%s and lang=%s",(word,lang))
    if res != []:
        res = res[0][0]
    return res

def trends(con, word, lang='nob', years=[]):
    res = []
    result = {}
    with con.cursor() as cur:
        cur.execute("select json from unigram where first=%s and lang=%s",(word,lang))
        res = cur.fetchall()
    result=dict()
    if res != []:
        res = res[0][0]
        if years != []:
            result = {key: res[key]/years[int(key)] for key in res.keys()}
        else:
            result = res
    return result

def trendsxy(con, words, lang='nob', smooth=3, years=[]):
    ws = words.split()
    res = {}
    for word in ws:
        jsn = trends(ngramcon, word, years=years)
        jsn = OrderedDict(sorted([(key,value) for key, value in jsn.items()]))
        if smooth > 0:
            jsny = smooth_values(list(jsn.values()), smooth)
        else:
            jsny = list(jsn.values())
        jsnx = list(jsn.keys())
        res[word] = {'x': jsnx, 'y': jsny}
    return res

In [20]:
from itertools import cycle

lst = "red blue green darkred lightblue".split()

cols = cycle(lst)


In [21]:
tdata = trendsxy(ngramcon, "abort grasrota", years=years)

NameError: name 'years' is not defined

In [ ]:
tdata = trendsxy(ngramcon, "abort grasrota", years=years)
p = figure(plot_width=900, plot_height=400)
p.legend.location = "bottom_left"
p.xgrid.grid_line_color = None
colorspectrum = cycle("darkblue darkred olive".split())
for key in tdata.keys():
    p.line(tdata[key]['x'], tdata[key]['y'], legend = key, line_width=2, line_color=next(colorspectrum))
show(p)

In [22]:
p = figure(plot_width=900, plot_height=400, webgl=True)
p.legend.location = "bottom_left"
p.xgrid.grid_line_color = None
colorspectrum = cycle(bp.Spectral9)
for key in tdata.keys():
    p.line(tdata[key]['x'], tdata[key]['y'], legend = key, line_width=2, line_color=next(colorspectrum))
show(p)

NameError: name 'figure' is not defined

In [ ]:
Counter(allc).most_common(2000)

In [ ]:
values = allc.values()
values = list(values)

In [ ]:
values = [x for x in values if x < 60]

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 9, 5

In [ ]:
n, m, c= plt.hist(values, 50)

In [ ]:

Counter(following(2008090304011)).most_common(20)

In [ ]:
%%timeit
finddb(2008090304011)

In [ ]:
urn2 = Counter(propers(2008090304011))
uni = Counter(unigram(2008090304011))

In [ ]:
print(urn2['To'], uni['To'])

In [ ]:
inter = urn2 & uni


In [ ]:
uni - urn2

In [ ]:
Counter(urn2).most_common(5)

In [ ]:
urn2.most_common(5)

In [ ]:
urn.most_common(10)

In [32]:
for row in find_books(tittel='Haiene', navn='%bjørneboe%')[:10]:
    print(row[0],finddb(row[0]))

NameError: name 'sqlexe' is not defined

In [ ]:
sqlpg('mods', "select * from mods_new where creator like 'Lefevre%'")

In [ ]:
sqlps('mods', "select * from mods_new where creator like 'Bjørneboe%' and title ")

In [ ]:
cur = geocon.cursor()
cur.execute('select * from stedsinfo')
[desc for desc in cur.description]

In [ ]:
sqldb('geo', "select * from stedsinfo where enh_snavn = 'Jostedalen'")

In [ ]:
sqldb('geo', "select sted.enh_snavn, array_agg(sted.kode), count(sted.enh_navntype) as c from stedsinfo as sted, unigram as uni \
where uni.urn = '2013030608218' and sted.enh_snavn = uni.word group by enh_snavn order by c desc")

In [ ]:
finddb(2013030608218)

In [ ]:
sqlexe('ft-7', "select word, freq from ft")

In [ ]:
dbs = ['ft-%s' % (str(i),) for i in range(9)]


In [ ]:
def avstand(sted1, sted2):
    cur = geocon.cursor()
    sql = "SELECT gg1.enh_snavn as stedsnavn1, gg1.wkb_geometry as sted1, \
    gg2.enh_snavn as stedsnavn2, gg2.wkb_geometry as sted2, ST_Distance_Sphere(gg1.wkb_geometry, gg2.wkb_geometry) as distanse \
    FROM (SELECT enh_snavn, wkb_geometry FROM stedsinfo WHERE enh_snavn ilike '%s') as gg1, \
    (SELECT enh_snavn, wkb_geometry FROM stedsinfo WHERE enh_snavn ilike '%s') as gg2" % (sted1, sted2)
    cur.execute(sql)
    return cur.fetchall()

In [ ]:
sorted(avstand('trondheim','bergen'),  key= lambda x: x[4])

In [24]:
import html

In [25]:
html.entities.codepoint2name

{34: 'quot',
 38: 'amp',
 60: 'lt',
 62: 'gt',
 160: 'nbsp',
 161: 'iexcl',
 162: 'cent',
 163: 'pound',
 164: 'curren',
 165: 'yen',
 166: 'brvbar',
 167: 'sect',
 168: 'uml',
 169: 'copy',
 170: 'ordf',
 171: 'laquo',
 172: 'not',
 173: 'shy',
 174: 'reg',
 175: 'macr',
 176: 'deg',
 177: 'plusmn',
 178: 'sup2',
 179: 'sup3',
 180: 'acute',
 181: 'micro',
 182: 'para',
 183: 'middot',
 184: 'cedil',
 185: 'sup1',
 186: 'ordm',
 187: 'raquo',
 188: 'frac14',
 189: 'frac12',
 190: 'frac34',
 191: 'iquest',
 192: 'Agrave',
 193: 'Aacute',
 194: 'Acirc',
 195: 'Atilde',
 196: 'Auml',
 197: 'Aring',
 198: 'AElig',
 199: 'Ccedil',
 200: 'Egrave',
 201: 'Eacute',
 202: 'Ecirc',
 203: 'Euml',
 204: 'Igrave',
 205: 'Iacute',
 206: 'Icirc',
 207: 'Iuml',
 208: 'ETH',
 209: 'Ntilde',
 210: 'Ograve',
 211: 'Oacute',
 212: 'Ocirc',
 213: 'Otilde',
 214: 'Ouml',
 215: 'times',
 216: 'Oslash',
 217: 'Ugrave',
 218: 'Uacute',
 219: 'Ucirc',
 220: 'Uuml',
 221: 'Yacute',
 222: 'THORN',
 223: 'szlig',

In [375]:
html.entities.name2codepoint

{'AElig': 198,
 'Aacute': 193,
 'Acirc': 194,
 'Agrave': 192,
 'Alpha': 913,
 'Aring': 197,
 'Atilde': 195,
 'Auml': 196,
 'Beta': 914,
 'Ccedil': 199,
 'Chi': 935,
 'Dagger': 8225,
 'Delta': 916,
 'ETH': 208,
 'Eacute': 201,
 'Ecirc': 202,
 'Egrave': 200,
 'Epsilon': 917,
 'Eta': 919,
 'Euml': 203,
 'Gamma': 915,
 'Iacute': 205,
 'Icirc': 206,
 'Igrave': 204,
 'Iota': 921,
 'Iuml': 207,
 'Kappa': 922,
 'Lambda': 923,
 'Mu': 924,
 'Ntilde': 209,
 'Nu': 925,
 'OElig': 338,
 'Oacute': 211,
 'Ocirc': 212,
 'Ograve': 210,
 'Omega': 937,
 'Omicron': 927,
 'Oslash': 216,
 'Otilde': 213,
 'Ouml': 214,
 'Phi': 934,
 'Pi': 928,
 'Prime': 8243,
 'Psi': 936,
 'Rho': 929,
 'Scaron': 352,
 'Sigma': 931,
 'THORN': 222,
 'Tau': 932,
 'Theta': 920,
 'Uacute': 218,
 'Ucirc': 219,
 'Ugrave': 217,
 'Upsilon': 933,
 'Uuml': 220,
 'Xi': 926,
 'Yacute': 221,
 'Yuml': 376,
 'Zeta': 918,
 'aacute': 225,
 'acirc': 226,
 'acute': 180,
 'aelig': 230,
 'agrave': 224,
 'alefsym': 8501,
 'alpha': 945,
 'amp': 38,
 

In [335]:
myhtml = dict(map(lambda x: (x[1], x[0]), list(html.entities.entitydefs.items())))